# This is a simple handwritting recognition example with conv-net to show how to use tensorboard

# Importing libraries

In [1]:
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data
import os

# Defining models placeholders

In [2]:
x = tf.placeholder(tf.float32, shape=[None, 784],name = "Image")
y_ = tf.placeholder(tf.float32, shape=[None, 10],name = "Correct_Image_label")

#Reshaping the input for convnet
x_image = tf.reshape(x, [-1,28,28,1],name = "Reshaped_Image")

# Definning some important functions

In [3]:
def weight_variable(shape):
  initial = tf.truncated_normal(shape, stddev=0.1)
  return tf.Variable(initial)

def bias_variable(shape):
  initial = tf.constant(0.1, shape=shape)
  return tf.Variable(initial)

In [4]:
def conv2d(x, W):
  return tf.nn.conv2d(x, W, strides=[1, 1, 1, 1], padding='SAME')

def max_pool_2x2(x):
  return tf.nn.max_pool(x, ksize=[1, 2, 2, 1],
                        strides=[1, 2, 2, 1], padding='SAME')

# Definning variables

In [5]:
#Variables for first convolutional layers
W_conv1 = weight_variable([5, 5, 1, 32])
b_conv1 = bias_variable([32])

#Variables for second convolutional layers
W_conv2 = weight_variable([5, 5, 32, 64])
b_conv2 = bias_variable([64])

#Variables for first fully connected layers
W_fc1 = weight_variable([7 * 7 * 64, 1024])
b_fc1 = bias_variable([1024])

#Variables for final fully connected layers
W_fc2 = weight_variable([1024, 10])
b_fc2 = bias_variable([10])

# Adding the layer of the graphs

In [6]:
# First convolutional layer
h_conv1 = tf.nn.relu(conv2d(x_image, W_conv1) + b_conv1)

# First pooling layer
h_pool1 = max_pool_2x2(h_conv1)

In [7]:
# Second convolutional layer
h_conv2 = tf.nn.relu(conv2d(h_pool1, W_conv2) + b_conv2)

# Second pooling layer
h_pool2 = max_pool_2x2(h_conv2)

In [8]:
# Flattening the final pooled layer
h_pool2_flat = tf.reshape(h_pool2, [-1, 7*7*64])

#First fully connected layer
h_fc1 = tf.nn.relu(tf.matmul(h_pool2_flat, W_fc1) + b_fc1)

In [9]:
# Introducing dropoutacc
keep_prob = tf.placeholder(tf.float32)
h_fc1_drop = tf.nn.dropout(h_fc1, keep_prob)

In [10]:
# Final softmax output
y_conv=tf.nn.softmax(tf.matmul(h_fc1_drop, W_fc2) + b_fc2)

### Loss 

In [11]:
cross_entropy = tf.reduce_mean(-tf.reduce_sum(y_ * tf.log(y_conv), reduction_indices=[1]))

### Trainning step

In [12]:
#Trainning step
train_step = tf.train.AdamOptimizer(1e-4).minimize(cross_entropy)

### Accuracy

In [13]:
#Correct prediction
correct_prediction = tf.equal(tf.argmax(y_conv,1), tf.argmax(y_,1))

#Accuracy
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))


# Creating summery for log data

In [14]:
# Create a summary to monitor loss tensor
tf.scalar_summary("loss", cross_entropy)

# Create a summary to monitor accuracy tensor
tf.scalar_summary("accuracy", accuracy)

# Merge all summaries into a single op
merged_summary_op = tf.merge_all_summaries()

# Initializing the session variables

In [15]:
sess = tf.InteractiveSession()
sess.run(tf.initialize_all_variables())

# Defining the log directory where the summery will remian and also the summery writer

In [16]:
logs_path = os.path.join(os.getcwd(),"logdata")
summary_writer = tf.train.SummaryWriter(logs_path, graph=tf.get_default_graph())

# Loading the MNIST dataset

In [17]:
mnist = input_data.read_data_sets('MNIST_data', one_hot=True)

Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz


# Trainning and writting the log data

In [18]:
print "Please go to %s directory and write 'tensorboard --logdir logdata/' on terminal and hit enter"%os.getcwd()
print "Now open the browser and type http://0.0.0.0:6006 and now you can see the tensorboard"

try:
    for i in range(20000):

        #Getting the batch
        batch = mnist.train.next_batch(50)

        if i%100 == 0:
            train_accuracy = accuracy.eval(feed_dict={
                x:batch[0], y_: batch[1], keep_prob: 1.0})
            print("step %d, training accuracy %g"%(i, train_accuracy))

        #Running the trainning and getting the summery out
        _,summary = sess.run([train_step,merged_summary_op],feed_dict={x: batch[0], y_: batch[1], keep_prob: 0.5})

        #Writting the summerys
        summary_writer.add_summary(summary, i)

    print("test accuracy %g"%accuracy.eval(feed_dict={
        x: mnist.test.images, y_: mnist.test.labels, keep_prob: 1.0}))
    
except KeyboardInterrupt:
    pass

Please go to /home/shezan directory and write 'tensorboard --logdir logdata/' on terminal and hit enter
Now open the browser and type http://0.0.0.0:6006 and now you can see the tensorboard
step 0, training accuracy 0.02
step 100, training accuracy 0.82
step 200, training accuracy 0.94
step 300, training accuracy 0.94
step 400, training accuracy 0.98
step 500, training accuracy 0.94
step 600, training accuracy 1
step 700, training accuracy 0.94
step 800, training accuracy 0.86
step 900, training accuracy 1
step 1000, training accuracy 0.96
step 1100, training accuracy 1
